In [91]:
import numpy as np
import pandas as pd
import scipy.stats as stat
from scipy.optimize import newton_krylov
from scipy.integrate import dblquad
from itertools import combinations, chain, repeat

In [82]:
def linear_additive(x):
    term1 = 2*x[0]
    term2 = 3*x[1]
    return term1 + term2

In [9]:
def rx2rn(distpair_type, param1, param2, rxpair):
    
    # getting the inverse cdf of distribution 1
    if (distpair_type[0] == 'unif'):
        mu1 = (param1[1] + param1[0])/2
        std1 = (param1[1] - param1[0])/12**0.5
        inv_cdf1 = lambda x : param1[0] + (param1[1] - param1[0])*x
    elif (distpair_type[0] == 'norm'):
        mu1 = param1[0]
        std1 = param1[1]
        inv_cdf1 = lambda x : stat.norm.ppf(x, mu1, std1)
    elif (distpair_type[0] == 'triangle'):
        mu1 = (param1[0] + param1[1] + param1[2])/3
        std1 = (np.sqrt(param1[0]**2+param1[1]**2+param1[2]**2-param1[0]*param1[1]-param1[0]*param1[2]-param1[1]*param1[2]))/np.sqrt(18)
        mid1=(param1[2]-param1[0])/(param1[1]-param1[0])
        term11= (param1[1]-param1[0])*(param1[2]-param1[0])
        term21= (param1[1]-param[0])*(param1[1]-param1[2])
        inv_cdf1 = lambda x : ((param1[0]+np.sqrt(term11)*np.sqrt(x/1))*((x>=0).astype(int))*((x<mid1).astype(int)) + (param1[1]-np.sqrt(term21)*np.sqrt(1-x))*((x>=mid1).astype(int))*((x<1).astype(int)))
    elif (distpair_type[0] == 'lognorm'):
        mu1= param1[0]
        std1=param1[1]
        # compute associated normal
        cv=std1/mu1**2
        m = np.log(mu1/(np.sqrt(1+cv)))
        v = np.sqrt(np.log(1+cv))           
        inv_cdf1 = lambda x : stat.lognorm.ppf(x, scale=np.exp(m), s=v, loc=0)
    elif (distpair_type[0] == 'expo'):
        lamda= param1[0]
        mu1=1/lamda
        std1=1/(lamda**2)
        inv_cdf1 = lambda x : stat.expon.ppf(x, scale=mu1)
    elif (distpair_type[0] == 'gev'):
        mu=param1[0] #location
        sigma=param1[1] #scale
        k1=param1[2] #shape
        inv_cdf1 = lambda x : stat.genextreme.ppf(x,c=k1,scale=sigma,loc=mu);
        [mu1,std1] = stat.genextreme.stats(k1,scale=sigma,loc=mu);
        
    # getting the inverse cdf of distribution 2
    if (distpair_type[1] == 'unif'):
        mu2 = (param2[1] + param2[0])/2
        std2 = (param2[1] - param2[0])/12**0.5
        inv_cdf2 = lambda x : param2[0] + (param2[1] - param2[0])*x
    elif (distpair_type[1] == 'norm'):
        mu2 = param2[0]
        std2 = param2[1]
        inv_cdf2 = lambda x : stat.norm.ppf(x, mu2, std2)
    elif (distpair_type[1] == 'triangle'):
        mu2 = (param2[0] + param2[1] + param2[2])/3
        std2 = (np.sqrt(param2[0]**2+param2[1]**2+param2[2]**2-param2[0]*param2[1]-param2[0]*param2[2]-param2[1]*param2[2]))/np.sqrt(18)
        mid2=(param2[2]-param2[0])/(param2[1]-param2[0])
        term12= (param2[1]-param2[0])*(param2[2]-param2[0])
        term22= (param2[1]-param2[0])*(param2[1]-param2[2])
        inv_cdf2 = lambda x : ((param2[0]+np.sqrt(term12)*np.sqrt(x/1))*((x>=0).astype(int))*((x<mid2).astype(int)) + (param2[1]-np.sqrt(term22)*np.sqrt(1-x))*((x>=mid1).astype(int))*((x<1).astype(int)))
    elif (distpair_type[1] == 'lognorm'):
        mu2= param2[0]
        std2=param2[1]
        # compute associated normal
        cv=std2/mu2**2
        m = np.log(mu2/(np.sqrt(1+cv)))
        v = np.sqrt(np.log(1+cv))           
        inv_cdf2 = lambda x : stat.lognorm.ppf(x, scale=np.exp(m), s=v, loc=0)
    elif (distpair_type[1] == 'expo'):
        lamda= param2[0]
        mu2=1/lamda
        std2=1/(lamda**2)
        inv_cdf2 = lambda x : stat.expon.ppf(x, scale=mu2)
    elif (distpair_type[1] == 'gev'):
        mu=param2[0] #location
        sigma=param2[1] #scale
        k2=param2[2] #shape
        inv_cdf2 = lambda x : stat.genextreme.ppf(x,c=k2,scale=sigma,loc=mu);
        [mu2,std2] = stat.genextreme.stats(k2,scale=sigma,loc=mu);
    
    # bivariate standard normal distribution
    stdnorm2_pdf = lambda x1, x2 : np.exp(-1*(x1**2 + x2**2)/2.0)/(2.0*np.pi)
    
    # integral bound zmax=5.0, zmin = -5.0
    integrand = lambda x1, x2 : inv_cdf1(stat.norm.cdf(x1*np.sqrt(1-rxpair**2)+ rxpair*x2,0,1))*inv_cdf2(stat.norm.cdf(x2,0,1))*stdnorm2_pdf(x1, x2)
    # compute double integral of integrand with x1 ranging from -5.0 to 5.0 and x2 ranging from -5.0 to 5.0
    rn = (dblquad(integrand, -5.0, 5.0, lambda x : -5.0, lambda x : 5.0) - mu1*mu2)/(std1*std2)
    
    return rn

In [10]:
def rn2rx(distpair_type, param1, param2, rnpair):    
    fun = lambda r : (rnpair - rx2rn(distpair_type, param1, param2, r))
    # try to find point x where fun(x) = 0
    try:
        rx = newton_krylov(fun, rnpair, x_tol=1e-5)
    except:
        rx = rnpair
            
    return rx    

In [11]:
def map_2_cornorm(parameters, corr_mat):
    # store parameter info in a list
    param_info = list(parameters.values())
    
    corr_n = np.eye(corr_mat.shape[0], corr_mat.shape[1])
    for i in range(0, corr_mat.shape[0] - 1):
        for j in range(i+1, corr_mat.shape[0]):
            # input paramter info (lb, ub, ?, dist type)
            corr_n[i][j] = rn2rx([param_info[i][3], param_info[j][3]], [param_info[i][0], param_info[i][1], param_info[i][2]],[param_info[j][0], param_info[j][1], param_info[j][2]],corr_mat[i][j])
            # matrix is symmetrical
            corr_n[j][i] = corr_n[i][j]
    return corr_n

In [12]:
def n2x_transform(norm_vectors, parameters):
    # Transform from correlated standard normal to original distributions
    param_info = list(parameters.values())
    
    # 
    k = norm_vectors.shape[1]   
    x = np.zeros(norm_vectors.shape)

    for i in range(0, k):
        if param_info[i][3] == 'unif':
            lb = param_info[i][0]
            ub = param_info[i][1]

            x[:, i] = lb + (ub - lb)*stat.norm.cdf(norm_vectors[:, i],0,1)
        elif param_info[i][3] == 'norm':
            mu = param_info[i][0]
            std = param_info[i][1]

            x[:, i] = stat.norm.ppf(stat.norm.cdf(norm_vectors[:, i],0,1), mu, std)
        elif param_info[i][3] == 'triangle':
            a = param_info[i][0]
            b = param_info[i][1]
            c = param_info[i][2]
            mid = (c-a)/(b-a)
            term1 = (b-a)*(c-a)
            term2 = (b-a)*(b-c)
            x_norm = stat.norm.cdf(norm_vector[:, i],0,1)
            x[:, i] = (a+np.sqrt(term1)*np.sqrt(x_norm))*((x_norm >= 0).astype(int))*((x_norm < mid).astype(int))+(b-np.sqrt(term2)*np.sqrt((1-x_norm)))*((x_norm >= mid).astype(int))*((x_norm < 1).astype(int))
        elif param_info[i][3] == 'lognorm':
            mu = param_info[i][0]
            std = param_info[i][1]
            term1 = std/mu**2
            m = np.log(mu/(np.sqrt(1+term1)))
            v = np.sqrt(np.log(1+term1))
            x[:, i] = np.lognorm.ppf(stat.norm.cdf(norm_vectors[:, i],0,1), scale=np.exp(mu), s=std, loc=0)
        elif param_info[i][3] == 'expo':
            mu = param_info[i][0]
            x[:, i] = np.expon.ppf(stat.norm.cdf(norm_vectors[:, i],0,1), scale=mu)
        elif param_info[i][3] == 'gev':
            mu = param_info[i][0] # location
            sigma = param_info[i][1] # scale
            k = param_info[i][2] # shape
            x[:, i] = stat.genextreme.ppf(stat.norm.cdf(norm_vectors[:, i],0,1),c=k,scale=sigma,loc=mu)
        
    return x

In [25]:
# GVARS inputs
seed = 12345678
num_stars = 10
num_dir_samples = 50
delta_h = 0.1
ivars_scales = [0.1, 0.3, 0.5]
parameters = {'x1' : (0, 1, None, 'norm'),
              'x2' : (0, 1, None, 'norm')}
corr_mat = np.array([[1, 0.6], [0.6, 1]])
n_var = len(parameters)

In [26]:
cov_mat = map_2_cornorm(parameters, corr_mat)
cov_mat

array([[1. , 0.6],
       [0.6, 1. ]])

In [27]:
# Generate independent standard normal samples
# the amount of samples is the same as the amount of stars
U = np.random.multivariate_normal(np.zeros(n_var), np.eye(n_var), num_stars)
U

array([[ 0.04188231, -1.43650834],
       [-2.37744837,  0.77764303],
       [-1.10330162, -2.02919731],
       [-1.16747121,  0.1087427 ],
       [ 0.43522013,  0.30987233],
       [ 2.17770596,  0.58895914],
       [-0.1934313 ,  1.12429862],
       [ 0.90325226, -0.30493676],
       [-0.21358142, -0.94669055],
       [ 0.06661068,  1.17500076]])

In [28]:
# Generate correlated standard normal samples
# the amount of samples is the same as the amount of stars
cholU = np.linalg.cholesky(cov_mat)
cholU = cholU.transpose() # to get in correct format for matrix multiplication
Z = np.matmul(U,cholU) # transform samples to standard normal distribution
display(Z)

array([[ 0.04188231, -1.12407728],
       [-2.37744837, -0.8043546 ],
       [-1.10330162, -2.28533882],
       [-1.16747121, -0.61348857],
       [ 0.43522013,  0.50902995],
       [ 2.17770596,  1.77779089],
       [-0.1934313 ,  0.78338012],
       [ 0.90325226,  0.29800195],
       [-0.21358142, -0.88550129],
       [ 0.06661068,  0.97996701]])

In [29]:
# Generate Nstar actual multivariate samples X
X = n2x_transform(Z, parameters)
X

array([[ 0.04188231, -1.12407728],
       [-2.37744837, -0.8043546 ],
       [-1.10330162, -2.28533882],
       [-1.16747121, -0.61348857],
       [ 0.43522013,  0.50902995],
       [ 2.17770596,  1.77779089],
       [-0.1934313 ,  0.78338012],
       [ 0.90325226,  0.29800195],
       [-0.21358142, -0.88550129],
       [ 0.06661068,  0.97996701]])

In [30]:
# define index matrix of complement subset
compsub = np.empty([n_var, n_var-1])
for i in range (0, n_var):

    temp = np.arange(n_var)
    compsub[i] = np.delete(temp, i)   
compsub = compsub.astype(int)
compsub

array([[1],
       [0]])

In [31]:
# computer coditional variance and conditional expectation for each star center
chol_cond_std = []
std_cond_norm = []
mui_on_noti = np.zeros((len(Z), n_var))
for i in range(0, n_var):
    noti = compsub[i]
    # 2 dimensional or greater matrix case
    if (cov_mat[noti, noti].ndim >= 2):
        cond_std = cov_mat[i][i] - cov_mat[i,noti]*np.linalg.inv(cov_mat[noti, noti])*cov_mat[noti,i]
        chol_cond_std.append(np.linalg.cholesky(cond_std))
        std_cond_norm.append(con_std)
        for j in range(0, len(Z)):
            mui_on_noti[j][i] = cov_mat[i,noti]*np.linalg.inv(cov_mat[noti, noti])*Z[j,noti]
    # less then 2 dimenional matrix case
    else:
        cond_std = cov_mat[i][i] - cov_mat[i,noti]*cov_mat[noti, noti]*cov_mat[noti,i]
        chol_cond_std.append(np.linalg.cholesky([[cond_std]]).flatten())
        std_cond_norm.append(cond_std)
        for j in range(0, len(Z)):
            mui_on_noti[j][i] = cov_mat[i, noti]*cov_mat[noti, noti]*Z[j, noti]


In [32]:
# Generate directional sample:
# Create samples in correlated standard normal space
all_section_condZ = []
condZ = []
for j in range(0, num_dir_samples):
    stnrm_base = np.random.multivariate_normal(np.zeros(n_var), np.eye(n_var), num_stars)
    for i in range(0, n_var):
        condZ.append(stnrm_base[:, i]*chol_cond_std[i] + mui_on_noti[:, i])
    all_section_condZ.append(condZ.copy())
    condZ.clear()
    
np.array(all_section_condZ).shape

(50, 2, 10)

In [35]:
# transform to original distribution and compute response surface
Xi_on_Xnoti = []
tmp1 = []
Xi_on_Xnoti_and_Xnoti_temp = []
Xi_on_Xnoti_and_Xnoti = []
for j in range(0, num_dir_samples):
    for i in range(0, len(parameters)):
        tmp1.append(n2x_transform(np.array([all_section_condZ[j][i]]).transpose(), parameters).flatten())
        tmp2 = X.copy()
        tmp2[:, i] = tmp1[i]
        Xi_on_Xnoti_and_Xnoti_temp.append(tmp2.copy()) 
    # attatch results from tmp1 onto Xi_on_Xnoti and Xi_on_Xnoti_and_Xnoti
    Xi_on_Xnoti.append(tmp1.copy())
    tmp1.clear() # clear for next iteration
    Xi_on_Xnoti_and_Xnoti.append(Xi_on_Xnoti_and_Xnoti_temp.copy())
    Xi_on_Xnoti_and_Xnoti_temp.clear() # clear for next iteration
        
np.array(Xi_on_Xnoti).shape # check that shape is the same as all_section condZ

(50, 2, 10)

In [75]:
# Create Star points
params = [*parameters]
star_points = {}
points = {}
temp = np.zeros([num_dir_samples, len(parameters)])
for i in range(0, num_stars):
    for j in range(0, len(parameters)):
        for k in range(0, num_dir_samples):
            temp[k, :] = Xi_on_Xnoti_and_Xnoti[k][j][i]
        points[params[j]] = np.copy(temp)
    star_points[i] = points.copy()

In [80]:
star_points_df = pd.concat({key: pd.concat({k: pd.DataFrame(d) for k, d in value.items()}) for key, value in star_points.items()})
star_points_df.index.names=['centre', 'param', 'points']
star_points_df

0         1
centre param points                    
0      x1    0      -0.368952 -1.124077
             1      -0.795732 -1.124077
             2      -0.570933 -1.124077
             3      -0.315725 -1.124077
             4      -1.157375 -1.124077
...                       ...       ...
9      x2    45      0.066611  0.663996
             46      0.066611 -0.649220
             47      0.066611 -0.766905
             48      0.066611 -0.504398
             49      0.066611  1.446239

[1000 rows x 2 columns]

In [81]:
from tqdm.autonotebook import tqdm

def apply_unique(func, df, axis=1, *args, **kwargs):
    '''Apply a function to unique rows of a DataFrame
    for efficiency.'''
    
    tqdm.pandas(desc=func.__name__ + ' evaluation')

    applied_df = df.merge(df.drop_duplicates()
                         .assign(**{func.__name__: lambda x: x.progress_apply(func, axis=axis)}), 
                         how='left')
    applied_df.index = df.index
    
    return applied_df

<ipython-input-81-2b471e26cf18>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [86]:
model_name = linear_additive
df = apply_unique(model_name, star_points_df, axis=1)
df.index.names=['centre', 'param', 'points']
df

linear_additive evaluation:   0%|          | 0/1000 [00:00<?, ?it/s]

0         1  linear_additive
centre param points                                     
0      x1    0      -0.368952 -1.124077        -4.110137
             1      -0.795732 -1.124077        -4.963695
             2      -0.570933 -1.124077        -4.514097
             3      -0.315725 -1.124077        -4.003683
             4      -1.157375 -1.124077        -5.686982
...                       ...       ...              ...
9      x2    45      0.066611  0.663996         2.125209
             46      0.066611 -0.649220        -1.814440
             47      0.066611 -0.766905        -2.167495
             48      0.066611 -0.504398        -1.379974
             49      0.066611  1.446239         4.471940

[1000 rows x 3 columns]

In [92]:
# getting the paired values of each section based on `h`
tqdm.pandas(desc='building pairs')

pair_df = df[model_name.__name__].groupby(level=[0,1]).progress_apply(section_df, delta_h=delta_h)
pair_df.index.names = ['centre', 'param', 'h', 'pair_ind']
pair_df

building pairs:   0%|          | 0/20 [00:00<?, ?it/s]

0         1
centre param h   pair_ind                    
0      x1    0.1 (0, 1)   -4.110137 -4.963695
                 (1, 2)   -4.963695 -4.514097
                 (2, 3)   -4.514097 -4.003683
                 (3, 4)   -4.003683 -5.686982
                 (4, 5)   -5.686982 -5.402377
...                             ...       ...
9      x2    4.7 (1, 48)  -2.456485 -1.379974
                 (2, 49)   0.508859  4.471940
             4.8 (0, 48)  -1.032056 -1.379974
                 (1, 49)  -2.456485  4.471940
             4.9 (0, 49)  -1.032056  4.471940

[24500 rows x 2 columns]

In [88]:
# helper functions
def apply_unique(func, df, axis=1, *args, **kwargs):
    '''Apply a function to unique rows of a DataFrame
    for efficiency.'''

    applied_df = df.merge(df.drop_duplicates()
                         .assign(**{func.__name__: lambda x: x.apply(func, axis=axis)}), 
                         how='left')
    applied_df.index = df.index
    
    return applied_df
    
    
def scale(df, bounds, axis=1, *args, **kwargs):
    '''scale the sampled matrix
    bounds is a dict with ['ub', 'lb'] keys
    the values are lists of the upper and lower bounds
    of the parameters/variables/factors'''
    
    # numpy equivalent for math operations
    bounds_np = {key:np.array(value) for key,value in bounds.items()}
    
    if axis:
        return df * (bounds_np['ub'] - bounds_np['lb']) + bounds_np['lb']
    else:
        return df.T * (bounds_np['ub'] - bounds_np['lb']) + bounds_np['lb']
    
    
def pairs_h(iterable):
    '''gives the pairs of numbers considering their differences'''
    interval = range(min(iterable), max(iterable)-min(iterable))
    pairs  = {key+1:[j for j in combinations(iterable, 2) if np.abs(j[0]-j[1])==key+1] for key in interval}
    return pairs
    
    
def section_df(df, delta_h): # ***delta_h here is newly added*** July 6th, 2021 - Saman's comment
    '''gets the paired values of each section based on index'''
    pairs = pairs_h(df.index.get_level_values(-1))
    df_values = df.to_numpy()
    sample = pd.concat({h*delta_h:
                    pd.DataFrame.from_dict({str(idx_tup): [df_values[idx_tup[0]], df_values[idx_tup[1]]] for idx_tup in idx}, 'index') \
                      for h, idx in pairs.items()}) 

    return sample
    
    
# lambda functions
'''covariogram of each section'''
cov_section = lambda pair_cols, mu_star: (pair_cols.sub(mu_star, axis=0)[0] * pair_cols.sub(mu_star, axis=0)[1]).groupby(level=[0,1,2]).mean()

'''variogram over all sections'''
variogram = lambda pair_cols: 0.5*(pair_cols[0] - pair_cols[1]).pow(2).groupby(level=[1,2]).mean()

'''morris sensitivity measure equivalent evaluated over all sections'''
morris_eq = lambda pair_cols: ((pair_cols[1] - pair_cols[0]).abs().groupby(level=[1,2]).mean(), \
                               (pair_cols[1] - pair_cols[0]).groupby(level=[1,2]).mean())

'''covariogram over all sections'''
covariogram = lambda pair_cols, mu_overall: ((pair_cols - mu_overall)[0] * (pair_cols - mu_overall)[1]).groupby(level=[1,2]).mean()

'''expected covariogram over all sections'''
e_covariogram = lambda cov_section_all: cov_section_all.groupby(level=[1,2]).mean()

'''sobol (total order) sensitivity measure equivalent evaluated over all sections''' # new sobol added *** 6 July 2021
# sobol_eq = lambda gamma, ecov, variance: ((gamma + ecov) / variance).loc[:,1]
sobol_eq = lambda gamma, ecov, variance, delta_h: ((gamma + ecov) / variance)[:, delta_h] # new July 6, 2021



# ivars function
def ivars(variogram_array, scale, delta_h):
    '''generate Integrated Variogram Across a Range of Scales (IVARS)
    by approximating area using right trapezoids having width of `delta_h`
    and hights of variogram values'''
    num_h  = len(variogram_value.index.levels[-1].to_list())
    x_bench= np.arange(start=0, stop=delta_h*(num_h+1), step=delta_h)
    x_int  = np.arange(start=0, stop=(scale*10+1)/10, step=delta_h)

    # calculate interpolated values for both x (h) and y (variogram)
    if x_int[-1] < scale:
        x_int.append(scale)
    y_bench= [0] + variogram_array.to_list()

    y_int  = np.interp(x=x_int, xp=x_bench, fp=y_bench)
    
    # for loop for each step size to caluclate the area
    ivars = 0
    for i in range(len(x_int)-1):
        ivars += 0.5*(y_int[i+1] + y_int[i]) * (x_int[i+1] - x_int[i])

    return ivars

# alias
idx = pd.IndexSlice